<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/pawpularity_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm
!pip install wandb
!pip install albumentations

In [ ]:
import os
import gc
import copy
import time
import random
from PIL import Image

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import albumentations as A
from albumentations.pytorch import ToTensor
import torchvision.transforms as T

import joblib
from tqdm import tqdm
from collections import defaultdict

import timm

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
import wandb

try:
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient()
  api_key = user_secrets.get_secret('wandb_api')
  wandb.login(key=api_key)
  anony = None
except:
  anony = 'must'

In [ ]:
ROOT_DIR = '/content'
TRAIN_DIR = '/content/train'
TEST_DIR = 'content/test'

In [ ]:
CONFIG = dict(
    seed = 42,
    model_name = 'tf_efficientnet_b4_ns',
    train_batch_size = 16,
    valid_batch_size = 32,
    img_size = 512,
    epochs = 10,
    learning_rate = 1e-5,
    scheduler = 'CosineAnnealingWarmRestarts',
    min_lr = 1e-4,
    T_max = 20,
    T_0 = 20,
    warmup_epochs = 0,
    weight_decay = 1e-6,
    n_accumulate = 1,
    n_fold = 5,
    num_classes = 1,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    competition = 'PetFinder',
    wandb_kernel = 'deb'
)

In [ ]:
def set_seed(seed = 42):
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CONFIG['seed'])

In [ ]:
def get_train_file_path(id):
  return f'{TRAIN_DIR}/{id}.jpg'

df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['file_path'] = df['Id'].apply(get_train_file_path)

feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]

In [ ]:
def create_folds(df, n_s, n_grp=None):
  df['kfold'] = -1

  if n_grp is None:
    skf = KFold(n_splits=n_s, random_state=CONFIG['seed'])
    target = df['Pawpularity']
  else:
    skf = StratifiedKFold(n_splits=n_s, shuffle=True, random_state=CONFIG['seed'])
    df['grp'] = pd.cut(df['Pawpularity'], bins=n_grp, labels=False)
    target = df['grp']

  for fold_no, (t,v) in enumerate(skf.split(target, target)):
    df.loc[v, 'kfold'] = fold_no
    
    df = df.drop(columns=['grp'])
    return df
  
  df = create_folds(df, n_s=CONFIG['n_fold'], n_grp=14)

In [ ]:
class PawpularityDataset(Dataset):
  def __init__(self, root_dir, df, transforms=None):
    self.root_dir = root_dir
    self.df = df
    self.file_name = df['file_path'].values
    self.targets = df['Pawpularity'].values
    self.meta = df[feature_cols].values
    self.transforms = transforms

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    img_path = self.file_name[index]
    img = cv2.imread(img_path)  # 이미지 읽고
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # RGB color로 변환
    meta = self.meta[index, :]  # 그 인덱스의 모든 열
    targets = self.targets[index]

    if self.transforms:
      img = self.transforms(image=img)['image']

    return img, meta, targets

In [ ]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

data_transforms = {
    'train' : T.Compose([
                         T.Resize(CONFIG['img_size'], CONFIG['img_size']),
                         T.RandomHorizontalFlip(),
                         T.RandomVerticalFlip(),
                         T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                         T.ColorJitter(brighthness=0.1, contrast=0.1, saturation=0.1),
                         T.Normalize(mean=MEAN, std=STD)
    ]),

    'valid' : T.Compose([
                         T.Resize(CONFIG['img_size'], CONFIG['img_size']),
                         T.Normalize(mean=MEAN, std=STD)
    ])
}

data_transforms = {
    'train' : A.Compose([
                         A.Resize(CONFIG['img_size'], CONFIG['img_size']),
                         A.HorizontalFlip(),
                         A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
                         ToTensor()
    ], p=1.0),

    'valid' : A.Compose([
                         A.Resize(CONFIG['img_size'], CONFIG['img_size']),
                         A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
                         ToTensor()
    ], p=1.0)
}

In [ ]:
class PawpularityModel(nn.Module):
  def __init__(self, model_name, pretrained=True):
    super(PawpularityModel, self).__init__()
    self.model = timm.create_model(model_name=model_name, pretrained=pretrained)
    self.n_features = self.model.classifier.in_features
    self.model.reset_classifier(0)
    self.fc = nn.Linear(self.n_features + 12, CONFIG['num_classes'])
    self.dropout = nn.Dropout(p=0.6)

  def forward(self, images, meta):
    features = model(images)
    features = self.dropout(features)
    features = torch.cat([features, meta], dim=1)
    output = self.fc(features)
    return output

model = PawpularityModel(CONFIG['model_name'])
model.to(CONFIG['device'])

PawpularityModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, tra

In [ ]:
def criterion(outputs, targets):
  return torch.sqrt(nn.MSELoss()(outputs.view(-1), targets.view(-1)))

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
  model.train()
  scaler = amp.GradScaler()

  dataset_size = 0
  running_loss = 0.0

  bar = tqdm(enumerate(dataloader), total=len(dataloader))
  for step, (images, meta, targets) in bar:
    images = images.to(device, dtype=torch.float)
    meta = meta.to(device, dtype=torch.float)
    targets = targets.to(device, dtype=torch.float)

    batch_size = images.size(0)

    with amp.autocast(enabled=True):
      outputs = model(images, meta)
      loss = criterion(outputs, targets)
      loss = loss / CONFIG['n_accumulate']

    scaler.step(loss).backward()

    if (step+1) % CONFIG['n_accumulate'] == 0:
      scaler.step(optimizer)
      scaler.update()

      optimizer.zero_grad()

      if scheduler is not None:
        scheduler.step()

    running_loss += (loss.item() * batch_size)
    dataset_size += batch_size

    epoch_loss = running_loss / dataset_size

    bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])
  
  gc.collect()

  return epoch_loss

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
  model.eval()

  dataset_size=0
  running_loss=0.0

  TARGETS=[]
  PREDS=[]

  bar = tqdm(enumerate(dataloader), total=len(dataloader))
  for step, (images, meta, targets) in bar:
    images = images.to(device, dtype=torch.float)
    meta = meta.to(device, dtype=torch.float)
    targets = targets.to(device, dtype=torch.float)

    batch_size = images.size(0)

    outputs = model(images, meta)
    loss = criterion(outputs, targets)

    running_loss += (loss.item() * batch_size)
    dataset_size += batch_size

    epoch_loss = running_loss / dataset_size

    PREDS.append(outputs.view(-1).cpu().detach().numpy())
    TARGETS.append(targets.view(-1).cpu().detach().numpy())

    bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, LR=optimizer.param_groups[0]['lr'])

  TARGETS = np.concatenate(TARGETS)
  PREDS = np.concatenate(PREDS)
  val_rmse = mean_squared_error(TARGETS, PREDS, squared=False)
  gc.collect()

  return epoch_loss, val_rmse

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
  wandb.watch(model, log_freq=100)  # wandb 로그

  start = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_epoch_rmse = np.inf
  history = defaultdict(list)

  for epoch in range(1, num_epochs+1):
    gc.collect()
    train_epoch_loss = train_one_epoch(model, optimizer, scheduler, dataloader, device=CONFIG['device'], epoch=epochs)

    valid_epoch_loss, valid_epoch_rmse = valid_one_epoch(model, valid_loader, device=CONFIG['device'], epoch=epochs)

    history['Train Loss'].append(train_epoch_loss)
    history['Valid Loss'].append(valid_epoch_loss)
    history['Valid RMSE'].append(valid_epoch_rmse)

    # wandb 로그
    wandb.log({'Train Loss':train_epoch_loss})
    wandb.log({'Valid Loss':valid_epoch_loss})
    wandb.log({'Valid RMSE':valid_epoch_rmse})

    print(f'Valid RMSE : {valid_epoch_rmse}')

    if valid_epoch_rmse <= best_epoch_rmse:
      print(f'{c_}Validation Loss Improved ({best_epoch_rmse}) --> ({valid_epoch_rmse})')
      best_epoch_rmse = valid_epoch_rmse
      run.summary['Best RMSE'] = best_epoch_rmse
      best_model_wts = copy.deepcopy(model.state_dict())
      PATH = f'epoch{epoch}.bin'
      torch.save(model.state_dict(), PATH)
      wandb.save(PATH)
      print(f'Model Saved{sr_}')

  end = time.time()
  time_elapsed = end - start
  print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
  print("Best RMSE: {:.4f}".format(best_epoch_rmse))

  model.load_state_dict(best_model_wts)
  
  return model, history

In [ ]:
def prepare_loader(fold):
  df_train = df[df.kfold != fold].reset_index(drop=True)
  df_valid = df[df.kfold == fold].reset_index(drop=True)

  train_dataset = PawpularityDataset(TRAIN_DIR, df_train, transforms=data_transforms['train'])
  valid_dataset = PawpularityDataset(TRAIN_DIR, df_valid, transforms=data_transforms['valid'])

  train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], num_workers=4, shuffle=True, pin_memory=True, drop_last=True)
  valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], num_workers=4, shuffle=False, pin_memory=True)

  return train_loader, valid_loader

train_loader, valid_loader = prepare_loader(fold=0)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CONFIG['T_0'], eta_min=CONFIG['min_lr'])

In [ ]:
# wandb 모델링 시작할거란
run = wandb.init(project='Pawpularity',
                 config=CONFIG,
                 job_type='Train',
                 anonymous='must')

In [ ]:
model, history = run_training(model, optimizer, scheduler, device=CONFIG['device'], num_epochs=CONFIG['epochs'])

In [ ]:
# wandb run finish
run.finist

# Submission